# Random Forest Regressor 

## Import libraries


In [375]:
import numpy as np
import pandas as pd

## Import the dataset

In [376]:
raw_data = pd.read_csv('raw_flat_prices_data.csv', parse_dates= ['month', 'lease_commence_date'])

raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287196 entries, 0 to 287195
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   month                287196 non-null  datetime64[ns]
 1   town                 287196 non-null  object        
 2   flat_type            287196 non-null  object        
 3   block                287196 non-null  object        
 4   street_name          287196 non-null  object        
 5   storey_range         287196 non-null  object        
 6   floor_area_sqm       287196 non-null  float64       
 7   flat_model           287196 non-null  object        
 8   lease_commence_date  287196 non-null  datetime64[ns]
 9   resale_price         287196 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(6)
memory usage: 21.9+ MB


## Feature engineering and data preprocessing plan

* month - only use the year of transaction
* town - utilize Leave One Out Encoding due to large number of categories (26) and want to preserve information of price in relation to town
* flat_type - utilize one hot encoding (7 categories)
* block and street_name - will not be used in model for simplicity of the model
* storey_range - grouped into 4 groups (low, mid, high, very high) and then one hot encoded
* floor_area_sqm - used as a numeric value
* flat_model - will not be used in model for simplicity of model (Many of the models are under-represented)
* lease_commence_date - used as a numeric value
* resale price - used as a numeric value (target)


In [377]:
df_fe = raw_data[['month', 'town', 'flat_type', 'storey_range', 'floor_area_sqm', 'lease_commence_date', 'resale_price']].copy(deep= True)

df_fe

,month,town,flat_type,storey_range,floor_area_sqm,lease_commence_date,resale_price
0,1990-01-01,ANG MO KIO,1 ROOM,10 TO 12,31.0,1977-01-01,9000
1,1990-01-01,ANG MO KIO,1 ROOM,04 TO 06,31.0,1977-01-01,6000
2,1990-01-01,ANG MO KIO,1 ROOM,10 TO 12,31.0,1977-01-01,8000
3,1990-01-01,ANG MO KIO,1 ROOM,07 TO 09,31.0,1977-01-01,6000
4,1990-01-01,ANG MO KIO,3 ROOM,04 TO 06,73.0,1976-01-01,47200
...,...,...,...,...,...,...,...
287191,1999-12-01,YISHUN,EXECUTIVE,10 TO 12,142.0,1987-01-01,456000
287192,1999-12-01,YISHUN,EXECUTIVE,01 TO 03,142.0,1988-01-01,408000
287193,1999-12-01,YISHUN,EXECUTIVE,07 TO 09,146.0,1988-01-01,469000
287194,1999-12-01,YISHUN,EXECUTIVE,04 TO 06,146.0,1988-01-01,440000


In [378]:
df_fe['year_of_sale'] = df_fe['month'].dt.year

df_fe

,month,town,flat_type,storey_range,floor_area_sqm,lease_commence_date,resale_price,year_of_sale
0,1990-01-01,ANG MO KIO,1 ROOM,10 TO 12,31.0,1977-01-01,9000,1990
1,1990-01-01,ANG MO KIO,1 ROOM,04 TO 06,31.0,1977-01-01,6000,1990
2,1990-01-01,ANG MO KIO,1 ROOM,10 TO 12,31.0,1977-01-01,8000,1990
3,1990-01-01,ANG MO KIO,1 ROOM,07 TO 09,31.0,1977-01-01,6000,1990
4,1990-01-01,ANG MO KIO,3 ROOM,04 TO 06,73.0,1976-01-01,47200,1990
...,...,...,...,...,...,...,...,...
287191,1999-12-01,YISHUN,EXECUTIVE,10 TO 12,142.0,1987-01-01,456000,1999
287192,1999-12-01,YISHUN,EXECUTIVE,01 TO 03,142.0,1988-01-01,408000,1999
287193,1999-12-01,YISHUN,EXECUTIVE,07 TO 09,146.0,1988-01-01,469000,1999
287194,1999-12-01,YISHUN,EXECUTIVE,04 TO 06,146.0,1988-01-01,440000,1999


In [379]:
storey_class = {
    '01 TO 03': 'LOW',
    '04 TO 06': 'LOW',
    '07 TO 09': 'MID',
    '10 TO 12': 'MID',
    '13 TO 15': 'HIGH',
    '16 TO 18': 'HIGH',
    '19 TO 21': 'VERY HIGH',
    '22 TO 24': 'VERY HIGH',
    '25 TO 27': 'VERY HIGH'
}

In [380]:
storey_cat = [] 

for s in df_fe['storey_range']:
    storey_cat.append(storey_class[s])

df_fe['storey'] = storey_cat

df_fe

,month,town,flat_type,storey_range,floor_area_sqm,lease_commence_date,resale_price,year_of_sale,storey
0,1990-01-01,ANG MO KIO,1 ROOM,10 TO 12,31.0,1977-01-01,9000,1990,MID
1,1990-01-01,ANG MO KIO,1 ROOM,04 TO 06,31.0,1977-01-01,6000,1990,LOW
2,1990-01-01,ANG MO KIO,1 ROOM,10 TO 12,31.0,1977-01-01,8000,1990,MID
3,1990-01-01,ANG MO KIO,1 ROOM,07 TO 09,31.0,1977-01-01,6000,1990,MID
4,1990-01-01,ANG MO KIO,3 ROOM,04 TO 06,73.0,1976-01-01,47200,1990,LOW
...,...,...,...,...,...,...,...,...,...
287191,1999-12-01,YISHUN,EXECUTIVE,10 TO 12,142.0,1987-01-01,456000,1999,MID
287192,1999-12-01,YISHUN,EXECUTIVE,01 TO 03,142.0,1988-01-01,408000,1999,LOW
287193,1999-12-01,YISHUN,EXECUTIVE,07 TO 09,146.0,1988-01-01,469000,1999,MID
287194,1999-12-01,YISHUN,EXECUTIVE,04 TO 06,146.0,1988-01-01,440000,1999,LOW


In [381]:
df_fe['year_lease_commence'] = df_fe['lease_commence_date'].dt.year

df_fe

,month,town,flat_type,storey_range,floor_area_sqm,lease_commence_date,resale_price,year_of_sale,storey,year_lease_commence
0,1990-01-01,ANG MO KIO,1 ROOM,10 TO 12,31.0,1977-01-01,9000,1990,MID,1977
1,1990-01-01,ANG MO KIO,1 ROOM,04 TO 06,31.0,1977-01-01,6000,1990,LOW,1977
2,1990-01-01,ANG MO KIO,1 ROOM,10 TO 12,31.0,1977-01-01,8000,1990,MID,1977
3,1990-01-01,ANG MO KIO,1 ROOM,07 TO 09,31.0,1977-01-01,6000,1990,MID,1977
4,1990-01-01,ANG MO KIO,3 ROOM,04 TO 06,73.0,1976-01-01,47200,1990,LOW,1976
...,...,...,...,...,...,...,...,...,...,...
287191,1999-12-01,YISHUN,EXECUTIVE,10 TO 12,142.0,1987-01-01,456000,1999,MID,1987
287192,1999-12-01,YISHUN,EXECUTIVE,01 TO 03,142.0,1988-01-01,408000,1999,LOW,1988
287193,1999-12-01,YISHUN,EXECUTIVE,07 TO 09,146.0,1988-01-01,469000,1999,MID,1988
287194,1999-12-01,YISHUN,EXECUTIVE,04 TO 06,146.0,1988-01-01,440000,1999,LOW,1988


In [382]:
df_fe.drop(labels= ['month', 'storey_range', 'lease_commence_date'], axis= 1, inplace= True)

df_fe

,town,flat_type,floor_area_sqm,resale_price,year_of_sale,storey,year_lease_commence
0,ANG MO KIO,1 ROOM,31.0,9000,1990,MID,1977
1,ANG MO KIO,1 ROOM,31.0,6000,1990,LOW,1977
2,ANG MO KIO,1 ROOM,31.0,8000,1990,MID,1977
3,ANG MO KIO,1 ROOM,31.0,6000,1990,MID,1977
4,ANG MO KIO,3 ROOM,73.0,47200,1990,LOW,1976
...,...,...,...,...,...,...,...
287191,YISHUN,EXECUTIVE,142.0,456000,1999,MID,1987
287192,YISHUN,EXECUTIVE,142.0,408000,1999,LOW,1988
287193,YISHUN,EXECUTIVE,146.0,469000,1999,MID,1988
287194,YISHUN,EXECUTIVE,146.0,440000,1999,LOW,1988


## Training

In [383]:
from sklearn.model_selection import train_test_split

## Split data for training and testing

In [384]:
target = df_fe['resale_price'].copy(deep= True)
features = df_fe.drop('resale_price', axis= 1)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size= 0.3, random_state= 42)

X_train

,town,flat_type,floor_area_sqm,year_of_sale,storey,year_lease_commence
79762,TAMPINES,3 ROOM,60.0,1994,LOW,1985
59669,KALLANG/WHAMPOA,5 ROOM,114.0,1994,MID,1974
75897,PASIR RIS,5 ROOM,122.0,1994,MID,1989
216035,HOUGANG,3 ROOM,73.0,1998,MID,1978
249067,PASIR RIS,4 ROOM,106.0,1999,LOW,1995
...,...,...,...,...,...,...
119879,WOODLANDS,4 ROOM,83.0,1996,LOW,1990
259178,HOUGANG,3 ROOM,67.0,1999,MID,1983
131932,BEDOK,3 ROOM,83.0,1996,LOW,1980
146867,BEDOK,4 ROOM,92.0,1997,LOW,1978


## Further split the data for training and validation

In [385]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size= 0.3, random_state= 8)

X_train

,town,flat_type,floor_area_sqm,year_of_sale,storey,year_lease_commence
217170,PASIR RIS,4 ROOM,108.0,1998,LOW,1993
99114,TAMPINES,3 ROOM,74.0,1995,MID,1986
230916,CHOA CHU KANG,4 ROOM,104.0,1999,MID,1995
116779,TAMPINES,4 ROOM,93.0,1996,MID,1985
44766,GEYLANG,3 ROOM,62.0,1993,MID,1969
...,...,...,...,...,...,...
58535,BISHAN,5 ROOM,121.0,1994,HIGH,1987
12588,ANG MO KIO,3 ROOM,67.0,1991,LOW,1979
110081,JURONG EAST,4 ROOM,83.0,1995,MID,1986
188228,YISHUN,4 ROOM,92.0,1998,LOW,1984


In [386]:
## Ensure all categories in each column are represented

print('The number of unique towns in training set is:', len(X_train['town'].unique()))
print('The number of unique flat type in training set is:', len(X_train['flat_type'].unique()))
print('The number of unique storey is :', len(X_train['storey'].unique()))

The number of unique towns in training set is: 26
The number of unique flat type in training set is: 7
The number of unique storey is : 4


## Data preprocessing

In [387]:
from category_encoders import LeaveOneOutEncoder

town_looe = LeaveOneOutEncoder()

df_enc_town = town_looe.fit_transform(X_train['town'], y_train)

df_enc_town.rename(columns= {'town': 'town_enc'}, inplace= True)

df_enc_town

,town_enc
217170,375216.905164
99114,263305.387179
230916,302778.049227
116779,263294.168481
44766,184622.335731
...,...
58535,342257.574411
12588,163774.777294
110081,199860.331264
188228,215470.896466


In [388]:
X_train = pd.concat([X_train, df_enc_town], axis= 1)

X_train

,town,flat_type,floor_area_sqm,year_of_sale,storey,year_lease_commence,town_enc
217170,PASIR RIS,4 ROOM,108.0,1998,LOW,1993,375216.905164
99114,TAMPINES,3 ROOM,74.0,1995,MID,1986,263305.387179
230916,CHOA CHU KANG,4 ROOM,104.0,1999,MID,1995,302778.049227
116779,TAMPINES,4 ROOM,93.0,1996,MID,1985,263294.168481
44766,GEYLANG,3 ROOM,62.0,1993,MID,1969,184622.335731
...,...,...,...,...,...,...,...
58535,BISHAN,5 ROOM,121.0,1994,HIGH,1987,342257.574411
12588,ANG MO KIO,3 ROOM,67.0,1991,LOW,1979,163774.777294
110081,JURONG EAST,4 ROOM,83.0,1995,MID,1986,199860.331264
188228,YISHUN,4 ROOM,92.0,1998,LOW,1984,215470.896466


In [389]:
from sklearn.preprocessing import OneHotEncoder

flat_type_ohe = OneHotEncoder()

ind = X_train.index
flat_type_ohe_df = pd.DataFrame(flat_type_ohe.fit_transform(X_train['flat_type'].values.reshape(-1, 1)).toarray())

flat_type_ohe_df.columns = flat_type_ohe.get_feature_names_out(['flat_type'])

flat_type_ohe_df.index = ind

flat_type_ohe_df

,flat_type_1 ROOM,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI GENERATION
217170,0.0,0.0,0.0,1.0,0.0,0.0,0.0
99114,0.0,0.0,1.0,0.0,0.0,0.0,0.0
230916,0.0,0.0,0.0,1.0,0.0,0.0,0.0
116779,0.0,0.0,0.0,1.0,0.0,0.0,0.0
44766,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
58535,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12588,0.0,0.0,1.0,0.0,0.0,0.0,0.0
110081,0.0,0.0,0.0,1.0,0.0,0.0,0.0
188228,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [390]:
X_train = pd.concat([X_train, flat_type_ohe_df], axis= 1)

X_train

,town,flat_type,floor_area_sqm,year_of_sale,storey,year_lease_commence,town_enc,flat_type_1 ROOM,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI GENERATION
217170,PASIR RIS,4 ROOM,108.0,1998,LOW,1993,375216.905164,0.0,0.0,0.0,1.0,0.0,0.0,0.0
99114,TAMPINES,3 ROOM,74.0,1995,MID,1986,263305.387179,0.0,0.0,1.0,0.0,0.0,0.0,0.0
230916,CHOA CHU KANG,4 ROOM,104.0,1999,MID,1995,302778.049227,0.0,0.0,0.0,1.0,0.0,0.0,0.0
116779,TAMPINES,4 ROOM,93.0,1996,MID,1985,263294.168481,0.0,0.0,0.0,1.0,0.0,0.0,0.0
44766,GEYLANG,3 ROOM,62.0,1993,MID,1969,184622.335731,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58535,BISHAN,5 ROOM,121.0,1994,HIGH,1987,342257.574411,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12588,ANG MO KIO,3 ROOM,67.0,1991,LOW,1979,163774.777294,0.0,0.0,1.0,0.0,0.0,0.0,0.0
110081,JURONG EAST,4 ROOM,83.0,1995,MID,1986,199860.331264,0.0,0.0,0.0,1.0,0.0,0.0,0.0
188228,YISHUN,4 ROOM,92.0,1998,LOW,1984,215470.896466,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [391]:
storey_ohe = OneHotEncoder()

ind = X_train.index
storey_ohe_df = pd.DataFrame(storey_ohe.fit_transform(X_train['storey'].values.reshape(-1, 1)).toarray())

storey_ohe_df.columns = storey_ohe.get_feature_names_out(['storey'])

storey_ohe_df.index = ind

storey_ohe_df

,storey_HIGH,storey_LOW,storey_MID,storey_VERY HIGH
217170,0.0,1.0,0.0,0.0
99114,0.0,0.0,1.0,0.0
230916,0.0,0.0,1.0,0.0
116779,0.0,0.0,1.0,0.0
44766,0.0,0.0,1.0,0.0
...,...,...,...,...
58535,1.0,0.0,0.0,0.0
12588,0.0,1.0,0.0,0.0
110081,0.0,0.0,1.0,0.0
188228,0.0,1.0,0.0,0.0


In [392]:
X_train = pd.concat([X_train, storey_ohe_df], axis= 1)

X_train

,town,flat_type,floor_area_sqm,year_of_sale,storey,year_lease_commence,town_enc,flat_type_1 ROOM,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI GENERATION,storey_HIGH,storey_LOW,storey_MID,storey_VERY HIGH
217170,PASIR RIS,4 ROOM,108.0,1998,LOW,1993,375216.905164,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
99114,TAMPINES,3 ROOM,74.0,1995,MID,1986,263305.387179,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
230916,CHOA CHU KANG,4 ROOM,104.0,1999,MID,1995,302778.049227,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
116779,TAMPINES,4 ROOM,93.0,1996,MID,1985,263294.168481,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
44766,GEYLANG,3 ROOM,62.0,1993,MID,1969,184622.335731,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58535,BISHAN,5 ROOM,121.0,1994,HIGH,1987,342257.574411,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12588,ANG MO KIO,3 ROOM,67.0,1991,LOW,1979,163774.777294,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
110081,JURONG EAST,4 ROOM,83.0,1995,MID,1986,199860.331264,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
188228,YISHUN,4 ROOM,92.0,1998,LOW,1984,215470.896466,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [393]:
X_train.drop(columns= ['town', 'flat_type', 'storey'], inplace= True)

X_train

,floor_area_sqm,year_of_sale,year_lease_commence,town_enc,flat_type_1 ROOM,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI GENERATION,storey_HIGH,storey_LOW,storey_MID,storey_VERY HIGH
217170,108.0,1998,1993,375216.905164,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
99114,74.0,1995,1986,263305.387179,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
230916,104.0,1999,1995,302778.049227,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
116779,93.0,1996,1985,263294.168481,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
44766,62.0,1993,1969,184622.335731,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58535,121.0,1994,1987,342257.574411,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12588,67.0,1991,1979,163774.777294,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
110081,83.0,1995,1986,199860.331264,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
188228,92.0,1998,1984,215470.896466,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
